In [1]:
import pandas as pd, re
import numpy as np
import getschools as gs

In [4]:
def get_df(group,topic="persistence"):
    
    ret = pd.read_csv("../clean/csv/collegeEntrancePersistence-"+group+".csv")
    
    for c in [
        #u'2009_10_entrance_pct',
              '2009_10_'+topic+'_pct',
       #u'2010_11_entrance_pct', 
              u'2010_11_'+topic+'_pct',
       #u'2011_12_entrance_pct', 
              u'2011_12_'+topic+'_pct',
       #u'2012_13_entrance_pct', 
              u'2012_13_'+topic+'_pct',
       #u'2013_14_entrance_pct', 
              u'2013_14_'+topic+'_pct']:
        
        ret[c] = pd.to_numeric(ret[c],errors="coerce")

    ret["school_code"] = ret["school_code"].astype(str).str.zfill(7)
    ret["district_code"] = ret["district_code"].astype(str).str.zfill(7)

    ret = gs.add_drg(ret)
    
    def four_year(row):
        non_nulls = 0
        avgs = []
        for c in ['2009_10_'+topic+'_pct',
                  '2010_11_'+topic+'_pct',
                  '2011_12_'+topic+'_pct',
                  '2012_13_'+topic+'_pct',
                  '2013_14_'+topic+'_pct']:
            
            if row[c] is None: 
                continue
                
            non_nulls += 1
            avgs += [row[c]]
                
        if len(avgs) == 0 or non_nulls == 0:
            return None
        
        return np.nanmedian(avgs)
                
    ret["four_y_percent"] = ret.apply(four_year,axis=1)        
    
    ret = ret[ret["four_y_percent"].notnull()]
    return ret

get_df("all",topic="entrance").head()

,district,district_code,school,school_code,2009_10_entrance_pct,2009_10_persistence_pct,2010_11_entrance_pct,2010_11_persistence_pct,2011_12_entrance_pct,2011_12_persistence_pct,2012_13_entrance_pct,2012_13_persistence_pct,2013_14_entrance_pct,2013_14_persistence_pct,group,drg,four_y_percent
0,Ansonia School District,0020011,Ansonia High School,0026111,67.1,88.0,61.0,87.2,62.1,85.2,58.0,78.2,59.4,78.5,all,H,61.0
1,Avon School District,0040011,Avon High School,0046111,81.8,96.9,84.3,95.2,86.4,95.0,86.8,94.2,88.2,95.0,all,B,86.4
2,Berlin School District,0070011,Berlin High School,0076111,75.7,94.4,83.5,92.3,81.7,93.2,82.5,95.8,88.1,92.0,all,D,82.5
3,Bethel School District,0090011,Bethel High School,0096111,78.3,89.4,81.1,90.7,77.9,92.8,70.6,91.3,77.7,85.6,all,D,77.9
4,Bloomfield School District,0110011,Bloomfield High School,0116111,66.0,79.4,76.3,84.9,66.2,74.5,68.2,84.1,73.2,81.1,all,G,68.2


In [5]:
def schools_df(topic="persistence"):
    all_df = get_df("all",topic=topic)
    all_df["race_ethnicity"] = "All"
    
    race_df = get_df("race",topic=topic)
    
    return pd.concat([race_df, all_df])

schools_df().to_csv("../clean/for_db/college_pers.csv",index=False)
schools_df(topic="entrance").to_csv("../clean/for_db/college_entrance.csv",index=False)


In [6]:
def drg_report(df,groups=[]):
    
    ret  = df.copy()
    
    ret = ret.groupby(["drg"] + groups).agg({
            "four_y_percent":np.nanmedian
        }).reset_index()
    
    return ret

drg_report(schools_df(),groups=["race_ethnicity"]).to_csv("../clean/for_db/drg_college_pers.csv",index=False)
drg_report(schools_df(topic="entrance"),groups=["race_ethnicity"]).to_csv("../clean/for_db/drg_college_entrance.csv",index=False)

In [ ]:
drg_report(get_df("race"),groups=["race_ethnicity"])

In [7]:
def state_report(df,groups=[]):
    
    ret  = df.copy()
    
    if len(groups) > 0:
        ret = ret.groupby(groups).agg({
                "four_y_percent":np.nanmedian
        }).reset_index()
    
    
    
    return ret#["four_y_percent"].median
state_report(schools_df(),groups=["race_ethnicity"]).to_csv("../clean/for_db/state_college_pers.csv",index=False)
state_report(schools_df(topic="entrance"),groups=["race_ethnicity"]).to_csv("../clean/for_db/state_college_entrance.csv",index=False)

#state_report(get_df("race"),groups=["race_ethnicity"])

In [ ]:
def full_df():
    
    school_race_df = get_df("race")
    school_all_df = get_df("all")
    school_all_df["race_ethnicity"] = "All"
    
    df = pd.concat([school_race_df,school_all_df])
    
    drg_df = drg_report(df,groups=["race_ethnicity"])
    
    state_df = state_report(df,groups=["race_ethnicity"])
    
    def drg_val(row):
        matches = drg_df[(drg_df["drg"] == row["drg"]) & (drg_df["race_ethnicity"] == row["race_ethnicity"])]
        if len(matches) != 1:
            return None
        
        #print matches.iloc[0]
        return matches.iloc[0]["four_y_percent"]
        
    def state_val(row):
        matches = state_df[state_df["race_ethnicity"] == row["race_ethnicity"]]
        if len(matches) != 1:
            return None
        
        #print matches.iloc[0]
        return matches.iloc[0]["four_y_percent"]

    df["drg_4y"] = df.apply(drg_val, axis=1)
    df["state_4y"] = df.apply(state_val, axis=1)

    
    df = df[["district_code","district","school_code","school",
             "race_ethnicity","four_y_percent","drg_4y","state_4y"]]
    
    df.columns = ["district_code","district","school_code","school",
             "race","school_rate","drg_rate","state_rate"]
    return df

full_df()#.sort_values(by="school")
    

In [ ]:
full_df().to_csv("../clean/for_db/college_pers.csv",index=False)